# **Dependencies installation and Google Colab project setup**

 **Install depedencies**

In [1]:
!pip install pydrive
!pip install pydub
!pip install soundfile

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

     |████████████████████████████████| 993kB 3.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built pydrive


**Files setup**

In [2]:
# https://drive.google.com/open?id=1e9EZ3ZA6k2Qij3EnmGVrMLSKo9hcCxSp
# https://drive.google.com/open?id=1aVE-RIMYgQLykn55nF8XzY82AJPVUiWi
# https://drive.google.com/open?id=1DjOTVdqhQWHA6MmYeogileqc8Y0kEZB6
# https://drive.google.com/open?id=1_GcDEd4cxxQUKJ4Nqoae63FHtnP5ndbh
# 1rtxU9VHllQSYoC7J9sxCm6KKf7UMsfvO // features 2GB
# 1PXV4iK6Lmh0ZDAgZMwIaFUTK3l68vP4D // feature 5 GB
filenames = ['features.hdf5', '1.wav','mono_1_stereo_LQ.wav','mono_1_stereo_HQ']
# https://drive.google.com/open?id=1Ae1WQspnvC2wNz2ywQ0238_E2SlZpaGr
ids = ['1PXV4iK6Lmh0ZDAgZMwIaFUTK3l68vP4D', '1_OmHV0rVNyfbzZgxnYzbgS-ozzJ-OlHo','1UCmJvf67RUsFgYzLljS4a3T6Q0N5EBRi','1aVE-RIMYgQLykn55nF8XzY82AJPVUiWi'] # ids of the corresponding files uploaded to google drive
for i in range(len(ids)):
  # Authenticate and create the PyDrive client.
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
#   link = 
#   fluff, id = link.split('=')
  print (ids[i]) # Verify that you have everything after '='
  downloaded = drive.CreateFile({'id':ids[i]}) 
  downloaded.GetContentFile(filenames[i]) 
  # os.environ['KMP_DUPLICATE_LIB_OK']='True'
  features_filename = filenames[0]
  input_filename = filenames[2]
  groundtruth_filenam = filenames[3]

1PXV4iK6Lmh0ZDAgZMwIaFUTK3l68vP4D
1_OmHV0rVNyfbzZgxnYzbgS-ozzJ-OlHo
1UCmJvf67RUsFgYzLljS4a3T6Q0N5EBRi
1aVE-RIMYgQLykn55nF8XzY82AJPVUiWi


# **Features extraction and train the model**


**Function definitions:**

In [3]:
#%%
import os
import soundfile as sf
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint,EarlyStopping
import h5py
from librosa.output import write_wav
from pydub import AudioSegment
import os
import datetime
from scipy.io.wavfile import write

#Feature extraction
def feature_extraction(x,fs):

	frame_length_s = 0.04 # window length in seconds
	frame_length = int(fs*frame_length_s) # 40ms window length in samples
	# set an overlap ratio of 50 %
	hop_length = frame_length//2

	# Compute STFT
	_,_,X = signal.stft(x, noverlap=hop_length, fs=fs,nperseg=frame_length)
	number_frequencies, number_time_frames = X.shape
	phaseInfo = np.angle(X)
	X = np.abs(X)

	# Segmentation
	sample_length_s = 0.5 # segment length in seconds
	sample_length = int(sample_length_s/frame_length_s) # ~1s in samples

	# Trim the frames that can't be fitted into the segment size
	trimmed_X = X[:, :-(number_time_frames%sample_length)]
	trimmed_phaseInfo = phaseInfo[:, :-(number_time_frames%sample_length)]

	# Segmentation (number of freqs x number of frames x number of segment x 1). The last dimension is 'channel'.
	features = trimmed_X.reshape((number_frequencies,sample_length,-1,1), order='F')
	# Transpose the feature to be in form (number of segment x number of freqs x number of frames x 1)
	return trimmed_phaseInfo,features.transpose((2,0,1,3))

#Converts stereo wav to mono wav
def file_process(file_path):
	folderpath, filename = os.path.split(file_path)
	sound = AudioSegment.from_wav(file_path)
	sound = sound.set_channels(1)
	sound.export(os.path.join(mono_folder,'mono_'+filename), format="wav")


	# #%% --------Spectrogram--------------
	# # The whole file spectrogram------------------
	# to_be_plotted = trimmed_X
		# duration = len(x)/float(fs)
	# freq_scale = np.linspace(0, fs / 2, to_be_plotted.shape[0])
	# timeframe_scale = np.linspace(0, duration, to_be_plotted.shape[1])

	# # plot spectrogram (amplitude only)
	# W = np.abs(to_be_plotted)
	# plt.figure()
	# plt.subplot(2,1,1)
	# plt.pcolormesh(timeframe_scale, freq_scale, np.log(W+0.00001))
	# plt.xlabel('Time (s)')
	# plt.ylabel('Frequency (Hz)')
	# plt.title('Entire file Spectrogram (log scale)')
	# plt.tight_layout()

	# # First batch spectrogram--------------
	# freq_scale = np.linspace(0, fs / 2, number_frequencies)
	# timeframe_scale = np.linspace(0, segment_length*frame_length_s, segment_length)
	# # plot spectrogram (amplitude only)
	# W = np.abs(features[50])
	# plt.subplot(2,1,2)
	# plt.pcolormesh(timeframe_scale, freq_scale, np.log(W+0.00001))
	# plt.xlabel('Time (s)')
	# plt.ylabel('Frequency (Hz)')
	# plt.title('First batch spectrogram (log scale)')
	# plt.tight_layout()
	# plt.show()

#%%---Convert to mp3------
# AudioSegment.from_wav(mono_filepath).export(mp3_filepath, format="mp3")
#%%---------Test extract features-----------


def save_features(X_train,X_test,y_train,y_test):

	with h5py.File(features_filename, 'w') as f:
		f.create_dataset('X_train', data=X_train)
		f.create_dataset('X_test', data=X_test)
		f.create_dataset('y_train', data=y_train)
		f.create_dataset('y_test', data=y_test)

#%% -----Read data-----------

def read_features():
	with h5py.File(features_filename, 'r') as f:
			X_train = f.get('X_train').value
			X_test = f.get('X_test').value
			y_train = f.get('y_train').value
			y_test = f.get('y_test').value
	return X_train, y_train, X_test, y_test


#%%---------CNN Model-------

def get_model(features_shape):
	input_shape = (features_shape[1],features_shape[2], 1)# (number of freqs x number of frames in a segment x number of channels)
	model = Sequential()
	model.add(Conv2D(32, (5, 5),
			input_shape=input_shape,
			activation = "relu",
			padding = "same"))
	# model.add(MaxPooling2D(pool_size=(4, 4)))
	model.add(Conv2D(64, (5, 5),
			activation = "relu",
			padding = "same"))
			
	model.add(Conv2D(1, (10, 10),
			activation = "relu",
			padding = "same"))

	adam = Adam(lr=0.0003)
	model.compile(optimizer=adam, loss='mean_absolute_error', metrics=['mean_absolute_error'])
	model.summary()
	return model

def get_model(features_shape):
	input_shape = (features_shape[1],features_shape[2], 1)# (number of freqs x number of frames in a segment x number of channels)
	model = Sequential()
	model.add(Conv2D(32, (5, 5),
			input_shape=input_shape,
			activation = "relu",
			padding = "same"))
	# model.add(MaxPooling2D(pool_size=(4, 4)))
	model.add(Conv2D(64, (5, 5),
			activation = "relu",
			padding = "same"))
			
	model.add(Conv2D(1, (10, 10),
			activation = "relu",
			padding = "same"))

	adam = Adam(lr=0.0003)
	model.compile(optimizer=adam, loss='mean_absolute_error', metrics=['mean_absolute_error'])
	model.summary()
	return model


def reconstruct(y,fs,model):    
	phaseInfo,feat = feature_extraction(y,fs)
	yhat = model.predict(feat)
	
	#------RECONSTRUCT THE AUDIO--------
	# Restore to the original shape
	yrec = yhat.transpose((1,2,0,3))
	yrec = yrec.reshape((yrec.shape[0],-1), order='F')

	# Save output file
	_, xrec = signal.istft(yrec, fs)
	write("output.wav",fs,xrec)
	print('Output without phase info was saved.')

	yrec = yrec * np.exp(1j*phaseInfo)
	# Save output file
	_, xrec = signal.istft(yrec, fs)
	write_wav("output_with_phase.wav",xrec,fs,norm = True)
	print('Output with phase info was saved.')


Using TensorFlow backend.


**Extract features**

In [0]:
# Read from file
 X_train, y_train, X_test, y_test = read_features()

# Extract features manually
# x1, fs = sf.read(groundtruth_filenam)
# _,groundtruth_features = feature_extraction(x1,fs)

# x2, fs = sf.read(input_filename)
# _ ,input_features = feature_extraction(x2,fs)

# X_train,X_test,y_train,y_test = train_test_split(input_features,groundtruth_features,test_size=0.2)
# save_features(X_train,X_test,y_train,y_test)



**Train the model:**

In [9]:
model = get_model(y_train.shape)

model_filename = 'SRCNN_{date:%Y-%m-%d %H:%M:%S}_best.h5'.format( date=datetime.datetime.now())
checkpoint = ModelCheckpoint(model_filename, monitor='val_loss', verbose=1, save_best_only=True,
                                 save_weights_only=False, mode='min')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
callbacks_list = [checkpoint,es]
model.fit(X_train, y_train, batch_size=16, validation_data=(X_test, y_test),
                   shuffle=True, epochs=200, callbacks=callbacks_list)
model.save('test-{date:%Y-%m-%d %H:%M:%S}.h5'.format( date=datetime.datetime.now() ))

#%%

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 883, 12, 32)       832       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 883, 12, 64)       51264     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 883, 12, 1)        6401      
Total params: 58,497
Trainable params: 58,497
Non-trainable params: 0
_________________________________________________________________
Train on 24324 samples, validate on 9080 samples
Epoch 1/200
24324/24324 [==============================] - 92s 4ms/step - loss: 2.3657e-04 - mean_absolute_error: 2.3657e-04 - val_loss: 1.9624e-04 - val_mean_absolute_error: 1.9624e-04

Epoch 00001: val_loss improved from inf to 0.00020, saving model to SRCNN_2019-04-30 08:16:25_best.h5
Epoch 2/200
24324/24324 [==============================] - 91s 4

In [6]:

X_train.shape

(24324, 883, 12, 1)

# **Generate the output file**

---



In [10]:

#%% ----- PREDICT---------

# model = load_model('SRCNN_2019-04-23 18_47_20_best.h5')

y, fs = sf.read('1.wav')
reconstruct(y,fs,model)


Output without phase info was saved.
Output with phase info was saved.


In [0]:
model.save('model_106epochs.h5')

# **Play ground: Maxtrix transformation testing**

In [0]:

a1 = np.arange(128).reshape((4,-1))
a = a1
number_frequencies, number_time_frames = a.shape
a = a[:, :-(number_time_frames%3)]


b= a.reshape((number_frequencies,3,-1,1), order='F')
	
z = b.transpose((2,0,1,3))

NameError: ignored

In [0]:
d = z.transpose((1,2,0,3))
d = d.reshape((d.shape[0],-1),order='F')

In [0]:
phaseInfo.shape

(883, 13826)

In [0]:
d[0,:,:,0]

array([[  0,   3,   6,   9,  12,  15,  18,  21,  24,  27],
       [ 32,  35,  38,  41,  44,  47,  50,  53,  56,  59],
       [ 64,  67,  70,  73,  76,  79,  82,  85,  88,  91],
       [ 96,  99, 102, 105, 108, 111, 114, 117, 120, 123]])

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131304 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.3-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!ls '/content/drive/Colab Notebooks/mono_1_stereo_HQ.wav'

x1, fs = sf.read('/content/drive/Colab Notebooks/mono_1_stereo_HQ.wav')

'/content/drive/Colab Notebooks/mono_1_stereo_HQ.wav'


KeyboardInterrupt: ignored

In [0]:
frame_length_s = 0.04
fs = 44100
int(2**np.ceil(np.log2(fs*frame_length_s)))
int(fs*frame_length_s)

1764